# Notebook to build the Hebbian Learning module

In [4]:
import sys
print(sys.executable)
sys.path.insert(0,'C:\\Users\\Matthias\\OneDrive - UGent\\Documents\\DOCUMENTEN\\3. Thesis\\BSC\\')

c:\Users\Matthias\OneDrive - UGent\Documents\DOCUMENTEN\3. Thesis\BSC\bsc\Scripts\python.exe
insert performed


In [7]:
import jax
from jax import numpy as jnp
import flax
from flax import linen as nn

from bsc_utils.controller import ExplicitMLP

In [13]:
rng = jax.random.PRNGKey(0)

In [16]:
features = [3,5,2]
rng, rng_input = jax.random.split(rng, 2)
input = jax.random.normal(rng_input, (3,))
print(input)
joint_control = 'position'

[-2.2051446  -0.90678614 -0.51053447]


In [9]:
model = ExplicitMLP(features = features, joint_control = joint_control)

In [22]:
print(model)
rng, rng_init = jax.random.split(rng, 2)
params_init = model.init(rng_init, input)
print(params_init)
print(jax.tree_util.tree_map(lambda x: x.shape, params_init))

ExplicitMLP(
    # attributes
    features = [3, 5, 2]
    joint_control = 'position'
)
{'params': {'layers_0': {'kernel': Array([[ 0.01603832,  1.0514129 ,  0.4636761 ],
       [ 0.85403943,  0.494277  ,  0.7106874 ],
       [-0.31836075, -0.5612211 ,  0.0063866 ]], dtype=float32), 'bias': Array([0., 0., 0.], dtype=float32)}, 'layers_1': {'kernel': Array([[ 0.48150116, -0.56797975, -0.3696574 , -0.17230418, -0.5892921 ],
       [ 0.19058019,  0.8845992 ,  0.0779127 ,  0.48802233, -0.04629107],
       [-0.867175  ,  0.35683778, -0.17771758, -0.7615291 ,  0.2613028 ]],      dtype=float32), 'bias': Array([0., 0., 0., 0., 0.], dtype=float32)}, 'layers_2': {'kernel': Array([[-0.7612989 ,  0.247033  ],
       [-0.08448817,  0.5055236 ],
       [ 0.41171753,  0.11057541],
       [ 0.2146284 ,  0.10984545],
       [ 0.09837503,  0.24238424]], dtype=float32), 'bias': Array([0., 0.], dtype=float32)}}}
{'params': {'layers_0': {'bias': (3,), 'kernel': (3, 3)}, 'layers_1': {'bias': (5,), 'kernel':

In [25]:
hebbian_params_empty = jax.tree_util.tree_map(lambda x: jnp.zeros(jnp.concatenate(x.shape, jnp.array[1])), params_init)
print(jax.tree_util.tree_map(lambda x: x.shape, hebbian_params_empty))
                                              

TypeError: 'function' object is not subscriptable

In [ ]:
def hebbian_to_synapse_strength(input_node_activity, output_node_activity, hebbian_params):
    